# Speed comparison between HDF5 (NERSC curated dataset) and Xarr (Weatherbench data)

In [9]:
%run __common.py
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/iopsstor/scratch/cscs/stefschu/DSM500/github/modulus-a5275d8/modulus/distributed/manager.py:329: UserWarning: Distributed manager is already intialized
  warn("Distributed manager is already intialized")


```sh
cp /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1979.h5 /capstor/scratch/cscs/stefschu/
```

In [15]:
h5_path = "/capstor/scratch/cscs/stefschu/1979.h5"
xr_path = "/capstor/store/cscs/ERA5/weatherbench2_original/"

## File opening

In [24]:
%%timeit

with h5py.File(h5_path, "r") as f:
    h5_data = f["fields"]

431 μs ± 7.03 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%%timeit

ds = xr.open_zarr(xr_path)

1.79 s ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Data reading

In [49]:
%%timeit

ds = xr.open_zarr(xr_path)

variables = ["2m_temperature", "surface_pressure", "total_precipitation"]

time_index = 0

data = ds[variables].isel(time=slice(time_index, time_index + 8))
data = data.to_array().values
#print(type(data), data.__class__, data.shape, data.dtype, data.nbytes)

2.9 s ± 23.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%%timeit

with h5py.File(h5_path, "r") as f:
    h5_data = f["fields"]
    data = h5_data[time_index:time_index + 8, [3,5,8]]
# print(type(data), data.__class__, data.shape, data.dtype, data.nbytes)

    

20.3 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Data reading without file opening

In [51]:
ds = xr.open_zarr(xr_path)

variables = ["2m_temperature", "surface_pressure", "total_precipitation"]

time_index = 0

In [58]:
%%timeit

data = ds[variables].isel(time=slice(time_index, time_index + 8))
data = data.to_array().values

1.08 s ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
f = h5py.File(h5_path, "r")
h5_data = f["fields"]

In [57]:
%%timeit 

h5_data[time_index:time_index + 8, [3,5,8]]

19.1 ms ± 845 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
